# Read Source table

In [0]:
df = spark.table('workspace.bronze_pyspark.erp_cust_az12')
df.display()

# import methods & libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when, concat, coalesce, trim, lit, substring
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

## Explore Source Table

In [0]:
(df.select(col('GEN')).distinct()).display()

In [0]:
(df.select
 (
     (substring(
         col('CID'), 4, length(col('CID')))
         )
 )
 .where(col('CID').isNull()
        )).display()
#no null values

In [0]:
(df.select(col('BDATE')).where(col('BDATE').isNull())).display()
# No nulls

# Transformation

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, when, concat, coalesce, trim, lit, substring, regexp_replace,lead,abs, nullif,substring
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
df= spark.table('workspace.bronze_pyspark.erp_cust_az12')
cust_az =(df.select
(
    substring(
                col('CID'), 4, length(col('CID'))
                ).alias('customer_number'),
    col('BDATE').alias('birthdate'),
    when(upper(trim(col('GEN'))) == 'F', lit('Female'))
    .when(upper(trim(col('GEN'))) == 'M', lit('Male'))
    .when(upper(trim(col('GEN'))) == 'MALE', lit('Male'))
    .when(upper(trim(col('GEN'))) == 'FEMALE', lit('Female'))
    .otherwise(lit('n/a')).alias('gender')                
        
))


# Drop Target table if already Exists

In [0]:
spark.sql("""DROP TABLE IF EXISTS workspace.silver_pyspark.erp_cust_az""")

# Create Target tavle & load the transformed data into it

In [0]:
cust_az.write.format('delta').mode('overwrite').saveAsTable('workspace.silver_pyspark.erp_cust_az')

# Sanity Checks

In [0]:
df = spark.sql("""SELECT * FROM workspace.silver_pyspark.erp_cust_az """)
df.display()

In [0]:
(df.select(
    col('gender')
    ).distinct()).display()

In [0]:
%sql
select count(*) from workspace.silver_pyspark.erp_cust_az

# View table changes

In [0]:
spark.sql("""DESCRIBE HISTORY workspace.silver_pyspark.erp_cust_az""").display()